In [1]:
import yfinance as yf
import requests
import pandas as pd
import json
import warnings
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")

/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from datetime import datetime

def get_yfinance_statistics(quotes: list = ['AAPL']) -> pd.DataFrame:
    """
    Fetches Yahoo Finance key statistics for the given list of quotes.
    
    Args:
        quotes (list): A list of stock symbols to fetch data for.

    Returns:
        pd.DataFrame: A DataFrame containing the key statistics in long format with a Date column.
    """
    all_statistics = []
    current_date = datetime.now().strftime('%Y-%m-%d')  # Current date in YYYY-MM-DD format
    
    for quote in tqdm(quotes, desc="Fetching Yahoo Finance data"):
        url = f"https://finance.yahoo.com/quote/{quote}/key-statistics/"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extracting specific statistics
            for row in soup.select("table tbody tr"):
                # Extracting title and value
                columns = row.find_all("td")
                if len(columns) == 2:
                    key = columns[0].get_text(strip=True)
                    value = columns[1].get_text(strip=True)
                    all_statistics.append({'Date': current_date, 'Quote': quote, 'Statistic': key, 'Value': value})
        else:
            print(f"Failed to retrieve data for {quote}. HTTP Status Code: {response.status_code}")
    
    # Convert to a DataFrame
    df = pd.DataFrame(all_statistics)

    # Clean specific columns
    for col in ['% Held by Institutions', '% Held by Insiders']:
        mask = df['Statistic'] == col
        df.loc[mask, 'Value'] = df.loc[mask, 'Value'].str.replace('%', '').astype(float)

    return df

# Example usage
quotes = ["AAPL", "GOOG", "MSFT"]
statistics_df = get_yfinance_statistics(quotes)

# Save to a CSV file (optional)
statistics_df.to_csv('yfinance_statistics.csv', index=False)

# Display the resulting DataFrame
print(statistics_df)

# Example pivoting
pivoted_df = statistics_df.pivot(index='Statistic', columns=['Quote', 'Date'], values='Value')
print("\nPivoted DataFrame:")
print(pivoted_df)


Fetching Yahoo Finance data: 100%|██████████| 3/3 [00:03<00:00,  1.11s/it]

           Date Quote                   Statistic      Value
0    2024-12-16  AAPL            Fiscal Year Ends  9/28/2024
1    2024-12-16  AAPL  Most Recent Quarter  (mrq)  9/28/2024
2    2024-12-16  AAPL               Profit Margin     23.97%
3    2024-12-16  AAPL     Operating Margin  (ttm)     31.17%
4    2024-12-16  AAPL     Return on Assets  (ttm)     21.46%
..          ...   ...                         ...        ...
148  2024-12-16  MSFT               Payout Ratio4     24.77%
149  2024-12-16  MSFT              Dividend Date3  3/13/2025
150  2024-12-16  MSFT           Ex-Dividend Date4  2/20/2025
151  2024-12-16  MSFT          Last Split Factor2        2:1
152  2024-12-16  MSFT            Last Split Date3  2/18/2003

[153 rows x 4 columns]

Pivoted DataFrame:
Quote                                              AAPL        GOOG       MSFT
Date                                         2024-12-16  2024-12-16 2024-12-16
Statistic                                                         

In [3]:
statistics_df.pivot(index='Quote', columns='Statistic', values='Value')

Statistic,% Held by Insiders1,% Held by Institutions1,200-Day Moving Average3,5 Year Average Dividend Yield4,50-Day Moving Average3,52 Week High3,52 Week Low3,52 Week Range3,Avg Vol (10 day)3,Avg Vol (3 month)3,...,Shares Short (prior month 10/31/2024)4,Short % of Float (11/29/2024)4,Short % of Shares Outstanding (11/29/2024)4,Short Ratio (11/29/2024)4,Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Trailing Annual Dividend Rate3,Trailing Annual Dividend Yield3
Quote,,,,,,,,,,,,,,,,,,,,,
AAPL,2.06%,61.91%,208.95,0.62,232.34,250.80,164.08,26.67%,40.1M,47.29M,...,133.04M,1.02%,1.02%,3.41,65.17B,4.31,119.06B,209.06%,0.98,0.40%
GOOG,0.02%,61.43%,167.89,--,173.14,196.89,131.55,39.50%,22M,18.24M,...,40.44M,--,0.34%,2.03,93.23B,7.62,29.29B,9.32%,0.40,0.21%
MSFT,0.06%,73.51%,423.77,0.87,423.99,468.35,366.50,20.02%,20.29M,20.51M,...,62.52M,0.85%,0.85%,2.67,78.43B,10.55,96.84B,33.66%,3.08,0.69%


In [4]:
import json
# https://www.sec.gov/file/company-tickers
a = []
with open("symbol_list.json", "r") as f:
    df = json.load(f)
a = [i['ticker'] for j, i in df.items()]
len(a)

10016

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
def scrape_second_table():
    url = "https://finance.yahoo.com/u/yahoo-finance/watchlists/most-active-penny-stocks/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tables = soup.find_all('table')  # Get all tables on the page
        time.sleep(2)
        if len(tables) >= 2:  # Ensure at least two tables exist
            table = tables[1]  # Select the second table
            
            # Extract headers
            headers = [header.text for header in table.find_all('th')]
            
            # Extract rows
            rows = []
            for row in table.find_all('tr')[1:]:  # Skip header row
                cells = row.find_all('td')
                if len(cells) == len(headers):
                    rows.append([cell.text.strip() for cell in cells])
            
            # Create DataFrame
            df = pd.DataFrame(rows, columns=headers)
            return df
        else:
            print("Second table not found on the webpage.")
    else:
        print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

# Usage
df = scrape_second_table()

Second table not found on the webpage.


In [6]:
df['Last Price'] = df['Last Price'].astype(float)
list_of_active_penny_stocks = list(df[df['Last Price'] > 0.1]['Symbol'])
statistics_df = get_yfinance_statistics(list_of_active_penny_stocks)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
statistics_df[statistics_df['Statistic'].isin(['% Held by Institutions1', '% Held by Insiders1'])]
institution = statistics_df[statistics_df['Statistic'].isin(['% Held by Institutions1'])]
institution['Value'] = institution['Value'].apply(lambda x: 0 if x == "--" else float(x.replace("%", "")))
institution.sort_values(by='Value')

,Date,Quote,Statistic,Value
1109,2024-12-13,NIVF,% Held by Institutions1,0.00
197,2024-12-13,RZLV,% Held by Institutions1,0.00
512,2024-12-13,LAES,% Held by Institutions1,0.13
563,2024-12-13,TNXP,% Held by Institutions1,0.23
887,2024-12-13,LICN,% Held by Institutions1,0.43
989,2024-12-13,BURU,% Held by Institutions1,0.58
623,2024-12-13,WKEY,% Held by Institutions1,0.89
1160,2024-12-13,SYTA,% Held by Institutions1,1.00
1049,2024-12-13,UOKA,% Held by Institutions1,2.31
938,2024-12-13,TRUG,% Held by Institutions1,2.47
